In [4]:
from keras.models import load_model
import json
import cv2
from scipy import signal
import math
import numpy as np
from sklearn.metrics import confusion_matrix


def smooth1(x):
  X_new=x.reshape(228,)
  y=signal.savgol_filter(X_new,window_length=25, polyorder=5,deriv=1)
  y=np.expand_dims(y,1)
  y=np.asarray(y)
  return y
def smooth2(x):
  X_new=x.reshape(228,)
  y=signal.savgol_filter(X_new,window_length=25, polyorder=5,deriv=2)
  y=np.expand_dims(y,1)
  y=np.asarray(y)
  return y
def standardd(X):
	Xnew=np.asarray(X,dtype='float')
	total=0
	u=X.sum()/len(X)
	for i in range(len(X)):
		total+=((X[i]-u)**2)
	o=math.sqrt(total/len(X))
	for i in range(len(X)):
		Xnew[i]=float((X[i]-u)/o)
	return Xnew
def processdata(X_data):
  X_f=[]
  for i in range(X_data.shape[0]):
    X_data[i]=standardd(X_data[i])
    xtem=X_data[i].reshape(228,)
    X_f.append(cv2.resize(xtem,(1,224)))
  X_f=np.asarray(X_f)
  return X_f
def processdata1(X_data):
  for i in range(X_data.shape[0]):
    X_data[i]=standardd(X_data[i])
  X_f=[]
  X_tem=np.asarray(list(map(lambda e: smooth1(e),X_data)))
  for i in range(X_tem.shape[0]):
    xtem=X_tem[i].reshape(228,)
    X_f.append(cv2.resize(xtem,(1,224)))
  X_f=np.asarray(X_f)
  return X_f
def processdata2(X_data):
  X_f=[]
  for i in range(X_data.shape[0]):
    X_data[i]=standardd(X_data[i])
  X_tem=np.asarray(list(map(lambda e: smooth2(e),X_data)))
  for i in range(X_tem.shape[0]):
    xtem=X_tem[i].reshape(228,)
    X_f.append(cv2.resize(xtem,(1,224)))
  X_f=np.asarray(X_f)
  return X_f


testing = np.load('../testinghigh.npz')
X_data = testing['xdata']
y_true = testing['ytrue']



X_test0=processdata(X_data)
X_test1=processdata1(X_data)
X_test2=processdata2(X_data)


mymodel = load_model("../Model and Weight/modeleng2.h5")
with open('../label_dicteng1.json', 'r') as label_file:
    label_dict = json.loads(label_file.read())


y_pred=mymodel.predict([X_test0,X_test1,X_test2])
y_pred_o=[]
for i in range(y_pred.shape[0]):
  maxx=np.argmax(y_pred[i])
  y_pred_o.append(label_dict[maxx])
y_pred_o=np.asarray(y_pred_o)

cfs=confusion_matrix(y_true,y_pred_o)
print(cfs)
correct=0
for i in range(len(y_true)):
  if y_pred_o[i]==y_true[i]:
    correct+=1
print('test_acc: {0:.1f}%'.format(correct * 100/len(y_true)))

[[66  0  0  0  0]
 [ 0 66  0  0  0]
 [ 0  0 65  1  0]
 [ 0  0  1 65  0]
 [ 0  1  0  0 65]]
test_acc: 99.1%
